Copyright (c) 2023 Habana Labs, Ltd. an Intel Company.

#### Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Using Paramater Efficient Fine Tuning on Llama 2 with 7B Parameters on One Intel&reg; Gaudi&reg; 2 AI Accelerator
This example will Fine Tune the Llama2-7B model using Parameter Efficient Fine Tuining (PEFT) and then run inference on a text prompt.  This will be using the Llama2 model with two task examples from the Optimum Habana library on the Hugging Face model repository.   The Optimum Habana library is optimized for Deep Learning training and inference on First-gen Gaudi and Gaudi2 and offers tasks such as text generation, language modeling, question answering and more. For all the examples and models, please refer to the [Optimum Habana GitHub](https://github.com/huggingface/optimum-habana#validated-models).

This example will Fine Tune the Llama2-7B model using Parameter Efficient Fine Tuining (PEFT) on the timdettmers/openassistant-guanaco dataset using the Language-Modeling Task in Optimum Habana.

### Parameter Efficient Fine Tuning with Low Rank Adaptation
Parameter Efficient Fine Tuning is a strategy for adapting large pre-trained language models to specific tasks while minimizing computational and memory demands.   It aims to reduce the computational cost and memory requirements associated with fine-tuning large models while maintaining or even improving their performance.  It does so by adding a smaller task-specific layer, leveraging knowledge distillation, and often relying on few-shot learning, resulting in efficient yet effective models for various natural language understanding tasks.   PEFT starts with a pre-trained language model that has already learned a wide range of language understanding tasks from a large corpus of text data. These models are usually large and computationally expensive.   Instead of fine-tuning the entire pre-trained model, PEFT adds a task-specific layer or a few task-specific layers on top of the pre-trained model. These additional layers are relatively smaller and have fewer parameters compared to the base model.


In [1]:
%cd ~/Gaudi-tutorials/PyTorch/Single_card_tutorials

/home/ufa813565e6c2cd47e0e35a43b155dd4/Gaudi-tutorials/PyTorch/Single_card_tutorials


### Model Setup: 

##### Install the Parameter Efficient Fine Tuning Library methods
This is taking the PEFT method from the Hugging Face repository and will be used to help create the PEFT Fine Tuning with the Llama2 model.

In [2]:
import sys
!{sys.executable} -m pip install peft==0.10.0

Defaulting to user installation because normal site-packages is not writeable
  Using cached peft-0.10.0-py3-none-any.whl (199 kB)
  Using cached accelerate-1.1.1-py3-none-any.whl (333 kB)

[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


##### Install the Optimum-Habana Library

In [3]:
!{sys.executable} -m pip install -q optimum-habana==1.11.1


[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


##### Pull the Hugging Face Examples from GitHub
These contain the working Hugging Face Task Examples that have been optimized for Gaudi.  For Fine Tuning, we'll use the language-modeling task. 

In [4]:
%cd ~/Gaudi-tutorials/PyTorch/Single_card_tutorials

/home/ufa813565e6c2cd47e0e35a43b155dd4/Gaudi-tutorials/PyTorch/Single_card_tutorials


In [5]:
!git clone -b v1.11.1 https://github.com/huggingface/optimum-habana.git

Cloning into 'optimum-habana'...
remote: Enumerating objects: 19102, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 19102 (delta 54), reused 63 (delta 31), pack-reused 18995 (from 1)
Receiving objects: 100% (19102/19102), 9.54 MiB | 16.56 MiB/s, done.
Resolving deltas: 100% (13283/13283), done.
Note: switching to '989484c60c22a217dd5f89d9d70a703162188b7f'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false



##### Go to the Language Modeling Task and install the model specific requirements

In [6]:
%cd ~/Gaudi-tutorials/PyTorch/Single_card_tutorials/optimum-habana/examples/language-modeling

/home/ufa813565e6c2cd47e0e35a43b155dd4/Gaudi-tutorials/PyTorch/Single_card_tutorials/optimum-habana/examples/language-modeling


In [7]:
!pip install -q -r requirements.txt


[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip


##### How to access and Use the Llama 2 model

Use of the pretrained model is subject to compliance with third party licenses, including the “Llama 2 Community License Agreement” (LLAMAV2). For guidance on the intended use of the LLAMA2 model, what will be considered misuse and out-of-scope uses, who are the intended users and additional terms please review and read the instructions in this link https://ai.meta.com/llama/license/.
Users bear sole liability and responsibility to follow and comply with any third party licenses, and Habana Labs disclaims and will bear no liability with respect to users’ use or compliance with third party licenses.

To be able to run gated models like the Meta Llama model, you need the following: 
- Have a HuggingFace account
- Agree to the terms of use of the model in its model card on the HF Hub
- set a read token
- Login to your account using the HF CLI: run huggingface-cli login before launching your script

In [8]:
#!huggingface-cli login --token <your_token_here>

## Fine Tuning the model with PEFT and LoRA

We'll now run the fine tuning with the PEFT method. Remember that the PEFT methods only fine-tune a small number of extra model parameters, thereby greatly decreasing the computational and storage costs. Recent State-of-the-Art PEFT techniques achieve performance comparable to that of full fine-tuning.

##### Here's a summary of the command required to run the Fine Tuning, you'll run this in the next cell below. 
Note in this case the following: 
1. Using the language modeling with LoRA; `run_lora_clm.py`
2. It's very efficient: only 0.06% of the total paramters are being fine tuned of the total 7B parameters.
4. Only 3 epochs are needed for fine tuning, it takes less than 20 minutes to run with the openassisant-guanaco dataset.
5. We are using a cached model on the Intel Tiber Cloud to reduce download time.


In [9]:
!python3 run_lora_clm.py \
    --overwrite_output_dir=True \
    --model_name_or_path huggyllama/llama-7b \
    --dataset_name timdettmers/openassistant-guanaco \
    --bf16 True \
    --output_dir ~/Gaudi-tutorials/PyTorch/Single_card_tutorials/model_lora_llama_single \
    --num_train_epochs 3 \
    --per_device_train_batch_size 16 \
    --evaluation_strategy "no" \
    --save_strategy "no" \
    --learning_rate 1e-4 \
    --warmup_ratio  0.03 \
    --lr_scheduler_type "constant" \
    --max_grad_norm  0.3 \
    --logging_steps 1 \
    --do_train \
    --do_eval \
    --use_habana \
    --use_lazy_mode \
    --throughput_warmup_steps 3 \
    --lora_rank=8 \
    --lora_alpha=16 \
    --lora_dropout=0.05 \
    --lora_target_modules "q_proj" "v_proj" \
    --dataset_concatenation \
    --report_to none \
    --max_seq_length 512 \
    --low_cpu_mem_usage True \
    --validation_split_percentage 4 \
    --adam_epsilon 1e-08

/home/ufa813565e6c2cd47e0e35a43b155dd4/.local/lib/python3.10/site-packages/transformers/utils/generic.py:462: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/ufa813565e6c2cd47e0e35a43b155dd4/.local/lib/python3.10/site-packages/transformers/utils/generic.py:319: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/ufa813565e6c2cd47e0e35a43b155dd4/.local/lib/python3.10/site-packages/transformers/utils/generic.py:319: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
11/14/2024 05:39:16 - WARNING - __main__ -   Process rank: 0, device: hpu, distributed training: True, 16-bits training: True
11/14/2024 05:39:16 - INFO - __mai

#### LoRA Fine Tuning Completed
You will now see a "model_lora_llama_single" folder created which contains the PEFT model `adapter_model.bin` which will be used in the inference example below. 

## Inference with Llama 2

We'll now use the Hugging Face `text-generation` task to run inference on the Llama2-7b model; we'll generate text based on an included prompt.  Notice that we've included a path to the PEFT model that we've just created.

First, we'll move to the text-generation examples folder and install the requirements. 

In [4]:
%cd ~/Gaudi-tutorials/PyTorch/Single_card_tutorials/optimum-habana/examples/text-generation
!pip install -q -r requirements.txt

/home/ufa813565e6c2cd47e0e35a43b155dd4/Gaudi-tutorials/PyTorch/Single_card_tutorials/optimum-habana/examples/text-generation

[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip


You will see that we are now running inference with the `run_generation.py` task and we are including the PEFT model that we Fine Tuned in the steps above. 

```
--prompt "I am a dog. Please help me plan a surprise birthday party for my human, including fun activities, games and decorations. And don't forget to order a big bone-shaped cake for me to share with my fur friends!" \
--peft_model /root/Gaudi-tutorials/PyTorch/Single_card_tutorials/optimum-habana/examples/language-modeling/model_lora_llama_single
```

In [5]:
# Use the prompt above to compare between the PEFT and non-PEFT examples
prompt = input("Enter a prompt for text generation: ")

Enter a prompt for text generation:  we are in super computing conference


In [ ]:
cmd = f'python3 run_generation.py  --model_name_or_path huggyllama/llama-7b --batch_size 1 --do_sample --max_new_tokens 300 --n_iterations 4 \
      --use_hpu_graphs --use_kv_cache --bf16 --prompt "{prompt}" '
      
print(cmd)
import os
os.system(cmd)

python3 run_generation.py  --model_name_or_path huggyllama/llama-7b --batch_size 1 --do_sample --max_new_tokens 300 --n_iterations 4       --use_hpu_graphs --use_kv_cache --bf16 --prompt "we are in super computing conference" 


/home/ufa813565e6c2cd47e0e35a43b155dd4/.local/lib/python3.10/site-packages/transformers/utils/generic.py:462: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/ufa813565e6c2cd47e0e35a43b155dd4/.local/lib/python3.10/site-packages/transformers/utils/generic.py:319: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/ufa813565e6c2cd47e0e35a43b155dd4/.local/lib/python3.10/site-packages/transformers/utils/generic.py:319: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Fetching 2 files: 100%|██████████| 2/2 [00:00<00:00, 3990.77it/s]
11/14/2024 05:45:09 - INFO - __main__ - Single-device run.
/home/ufa813565e6c2cd47e0e35a43b155

Warming up
Warming up
Warming up


11/14/2024 05:46:07 - INFO - __main__ - Running generate...


##### Comparison without PEFT and LoRA
In this example, we're simply running the Llama2 7B model **without** including the PEFT fine tuned model, so the you are losing the additional detail that is brought to the model, and the results have signficantly less information and fidelity compared to the last model.

In [13]:
cmd = f'python3 run_generation.py  --model_name_or_path huggyllama/llama-7b --batch_size 1 --do_sample --max_new_tokens 300 --n_iterations 4 \
      --use_hpu_graphs --use_kv_cache --bf16 --prompt "{prompt}"'
print(cmd)
import os
os.system(cmd)

python3 run_generation.py  --model_name_or_path meta-llama/Llama-2-7b-hf --batch_size 1 --do_sample --max_new_tokens 300 --n_iterations 4       --use_hpu_graphs --use_kv_cache --bf16 --prompt "a red cat with red hat"


/home/ufa813565e6c2cd47e0e35a43b155dd4/.local/lib/python3.10/site-packages/transformers/utils/generic.py:462: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/ufa813565e6c2cd47e0e35a43b155dd4/.local/lib/python3.10/site-packages/transformers/utils/generic.py:319: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/ufa813565e6c2cd47e0e35a43b155dd4/.local/lib/python3.10/site-packages/transformers/utils/generic.py:319: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]
Traceback (most recent call last):
  File "/home/ufa813565e6c2cd47e0e35a43b155dd4/.local/lib/python3.10/

256

In [14]:
# Please be sure to run this exit command to ensure that the resorces running on Intel Gaudi are released 
exit()